In [1]:
import pandas as pd
import urllib.request
import json
import os
import ssl

In [2]:
#importing the data
true = pd.read_csv('true.csv')
fake = pd.read_csv('fake.csv')

#adding a column to each dataframe to indicate whether the news is fake or true
true['fake'] = 1
fake['fake'] = 0

#concatenating the two dataframes
data = pd.concat([true, fake])

#dropping the title column, date column, and subject column
data = data.drop(['title', 'date', 'subject'], axis=1)

#shuffling the data
data = data.sample(frac=1).reset_index(drop=True)

data.head()


,text,fake
0,Martin Luther King Jr. would not likely be ple...,0
1,In what seems to be another misspeak moment fo...,0
2,BARCELONA (Reuters) - Around one million Catal...,1
3,Republican Presidential front runner Donald Tr...,0
4,"Who is silencing political speech, physically...",0


In [3]:
input_data = data.copy()
input_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    44898 non-null  object
 1   fake    44898 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 701.7+ KB


In [4]:
# truncate the text column to 400 characters
data['text'] = data['text'].str.slice(0, 400)

In [5]:
# create a dictionary to store the text as input and the label as fake
data_dict = {'input': data['text'], 'fake': data['fake']}

# print the first 5 inputs of the dictionary
for i in range(5):
    print(data_dict['input'][i])
    print('-----------------')


Martin Luther King Jr. would not likely be pleased to see such a divisive figure using his position as a professional athlete to promote a violent and threatening movement funded and promoted by politicians to divide our nation Colin Kaepernick s football career remains very much a question mark, but he s already left an impression on American culture, as his jersey and other gear from his protest
-----------------
In what seems to be another misspeak moment for the Sanders  campaign, Bernie s campaign manager Jeff Weaver appeared on CNN and completely stuck his foot in his mouth for going after Hillary Clinton s ambition to be president. I mean, is she not supposed to be ambitious? Wouldn t ambition be a good quality in a leader? Someone that wants to get the job done?Weaver said: Don t destroy the Democrat
-----------------
BARCELONA (Reuters) - Around one million Catalans rallied in Barcelona on Monday, waving red and yellow striped flags and banging drums, in a show of support for 

In [6]:
data_dict['input'][1]

'In what seems to be another misspeak moment for the Sanders  campaign, Bernie s campaign manager Jeff Weaver appeared on CNN and completely stuck his foot in his mouth for going after Hillary Clinton s ambition to be president. I mean, is she not supposed to be ambitious? Wouldn t ambition be a good quality in a leader? Someone that wants to get the job done?Weaver said: Don t destroy the Democrat'

In [7]:
output = []
for i in range(1000):
    def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
        if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
            ssl._create_default_https_context = ssl._create_unverified_context

    allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

    # Request data goes here
    # The example below assumes JSON formatting which may be updated
    # depending on the format your endpoint expects.
    # More information can be found here:
    # https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
    data = {"inputs": data_dict['input'][i]}

    body = str.encode(json.dumps(data))

    url = 'https://bjr-ml-adcjv.canadacentral.inference.ml.azure.com/score'
    # Replace this with the primary/secondary key or AMLToken for the endpoint
    api_key = 'bLd614sjz7OTMV81zt6iJ4nvZetPCDHA'
    if not api_key:
        raise Exception("A key should be provided to invoke the endpoint")

    # The azureml-model-deployment header will force the request to go to a specific deployment.
    # Remove this header to have the request observe the endpoint traffic rules
    headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'jy46604790-fake-news-bert-det-3' }

    req = urllib.request.Request(url, body, headers)

    try:
        response = urllib.request.urlopen(req)

        result = response.read()
        result = result.decode('utf8').strip('[]')
        print(result)
        # output to a dictionary with index i
        output.append({i: result})



    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))

        # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
        print(error.info())
        print(error.read().decode("utf8", 'ignore'))


{"label":"LABEL_0","score":0.9992927312850952}
{"label":"LABEL_0","score":0.9996916055679321}
{"label":"LABEL_1","score":0.9999105930328369}
{"label":"LABEL_0","score":0.9993383288383484}
{"label":"LABEL_0","score":0.9990455508232117}
{"label":"LABEL_1","score":0.9999110698699951}
{"label":"LABEL_0","score":0.9980118274688721}
{"label":"LABEL_1","score":0.9999129772186279}
{"label":"LABEL_0","score":0.9996837377548218}
{"label":"LABEL_1","score":0.9999126195907593}
{"label":"LABEL_1","score":0.9999116659164429}
{"label":"LABEL_0","score":0.9996609687805176}
{"label":"LABEL_0","score":0.9987815022468567}
{"label":"LABEL_0","score":0.999667763710022}
{"label":"LABEL_1","score":0.9999123811721802}
{"label":"LABEL_0","score":0.9996858835220337}
{"label":"LABEL_0","score":0.99973064661026}
{"label":"LABEL_0","score":0.999679446220398}
{"label":"LABEL_1","score":0.9999127388000488}
{"label":"LABEL_0","score":0.9997097849845886}
{"label":"LABEL_0","score":0.99974125623703}
{"label":"LABEL_1",

In [8]:
# Extract and convert JSON strings to dictionaries
clean_data = []
for item in output:
  # Assuming the key is an integer (adjust if different)
  json_string = item[list(item.keys())[0]]  # Extract the JSON string
  clean_data.append(eval(json_string))  # Convert the JSON string to a dictionary

# reate a DataFrame from the cleaned data
output_df = pd.DataFrame(clean_data)

# encode the label column to 0 or 1 in output_df
output_df['label'] = output_df['label'].apply(lambda x: 1 if x == 'LABEL_1' else 0)

# Print the DataFrame
print(output_df)

print (output)

     label     score
0        0  0.999293
1        0  0.999692
2        1  0.999911
3        0  0.999338
4        0  0.999046
..     ...       ...
995      1  0.999910
996      0  0.999294
997      0  0.999271
998      1  0.999912
999      0  0.999637

[1000 rows x 2 columns]
[{0: '{"label":"LABEL_0","score":0.9992927312850952}'}, {1: '{"label":"LABEL_0","score":0.9996916055679321}'}, {2: '{"label":"LABEL_1","score":0.9999105930328369}'}, {3: '{"label":"LABEL_0","score":0.9993383288383484}'}, {4: '{"label":"LABEL_0","score":0.9990455508232117}'}, {5: '{"label":"LABEL_1","score":0.9999110698699951}'}, {6: '{"label":"LABEL_0","score":0.9980118274688721}'}, {7: '{"label":"LABEL_1","score":0.9999129772186279}'}, {8: '{"label":"LABEL_0","score":0.9996837377548218}'}, {9: '{"label":"LABEL_1","score":0.9999126195907593}'}, {10: '{"label":"LABEL_1","score":0.9999116659164429}'}, {11: '{"label":"LABEL_0","score":0.9996609687805176}'}, {12: '{"label":"LABEL_0","score":0.9987815022468567}'}, {13:

In [9]:
print(input_data['fake'][0:10])

0    0
1    0
2    1
3    0
4    0
5    1
6    0
7    1
8    0
9    1
Name: fake, dtype: int64


In [10]:
print(output_df['label'][0:10])

0    0
1    0
2    1
3    0
4    0
5    1
6    0
7    1
8    0
9    1
Name: label, dtype: int64


In [11]:
from sklearn.metrics import accuracy_score 

# calculate the accuracy of the model
accuracy = accuracy_score(input_data['fake'][0:1000], output_df['label'])
print(accuracy)

0.997
